## Just for 2007 - 2017 CDBG for now

In [9]:
import pandas as pd

In [225]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

In [10]:
path = '/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/'
df_house = pd.read_csv(path + 'ToDatabase/TotalHouse_v04.csv')

In [11]:
df_cdbg = pd.read_csv(path + 'Raw-Data/test_cdbg_2007-2017.csv')

In [12]:
df_house.drop(['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1)
df_cdbg.drop('Unnamed: 0', inplace=True, axis = 1)

In [13]:
df_cdbg.head()

,Address,Description,FundedAmount,IDISActivityID,IDISProjectId,InitialFundingDate,PlanYear,Project,Status,StatusDate
0,SANDS DRIVE,,0.00,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
1,421 W SOCIETY AVE,,124559.66,522,11,08/22/2007,2006,DEMOLITION,Completed,4/6/2010
2,525 5TH AVE,,15016.43,523,8,08/22/2007,2006,PUBLIC FACILITIES,Completed,8/16/2011
3,PO BOX 448,DAYCARE SERVICES FOR SENIORS DIAGNOSED WITH AL...,14950.95,539,15,10/04/2007,2007,PUBLIC SERVICE,Completed,8/6/2010
4,400 EBONY LN,Drainage and street improvements made to the 4...,46642.04,546,11,10/04/2007,2007,PUBLIC FACILITIES,Completed,11/29/2010


In [14]:
df_house.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
0,21204,0000Q/00025/009,1707 EDGERLY AVE,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0,NaN,NaN
1,26763,0000U/00006/002,703 DARTMOUTH LN,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0,NaN,NaN
2,21680,0000H/00047/005,1115 W WADDELL AVE,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0,NaN,NaN
3,28935,0000X/00003/034,313 FORRESTAL LN,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0,NaN,NaN
4,22320,00240/00003/024,1125 JOEL DR,2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0,NaN,NaN


In [15]:
s_cdbg_id = df_cdbg['IDISActivityID'] #pull ids
s_house_id = df_house['IDIS Activity ID'] #pull ids

In [17]:
s_cdbg_id.shape #shapes

(1111,)

In [18]:
s_house_id.shape #shapes

(850,)

#### We want to merge the two databases based upon id... we need to do some preliminary exploration first

In [20]:
s_cdbg_id.value_counts().head() # so multiple projects have the same id which is not what we anticipated

466     5
771     5
523     5
1146    5
547     4
Name: IDISActivityID, dtype: int64

In [22]:
s_cdbg_id.value_counts().tail()

1042    1
1043    1
1057    1
1096    1
445     1
Name: IDISActivityID, dtype: int64

In [21]:
df_cdbg[s_cdbg_id == 466] # but luckily these are just duplicate records

,Address,Description,FundedAmount,IDISActivityID,IDISProjectId,InitialFundingDate,PlanYear,Project,Status,StatusDate
0,SANDS DRIVE,,0.0,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
137,SANDS DRIVE,,0.0,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
405,SANDS DRIVE,,0.0,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
514,SANDS DRIVE,,0.0,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
644,SANDS DRIVE,,0.0,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012


#### Let's see what happens when we drop duplicate records

In [25]:
df_cdbg = df_cdbg.drop_duplicates()

In [26]:
s_cdbg_id = df_cdbg['IDISActivityID']

In [29]:
s_cdbg_id.value_counts().head() # so now each id has only one record attached to it

1023    1
1401    1
1426    1
1425    1
1424    1
Name: IDISActivityID, dtype: int64

#### Let's check the intersection of ids found in the pr03s drawdown and the ids found in the housing projects list

In [35]:
cdbg_notin_house = df_cdbg[~s_cdbg_id.isin(s_house_id)]

In [37]:
house_notin_cdbg = df_house[~s_house_id.isin(s_cdbg_id)] # these are the more worrisome ones

#### how many of these are home grants? Let's try this again, but after filtering for only cdbg in our dataset

In [58]:
df_house_cdbg = df_house[df_house['Program'] == "CDBG"]

In [59]:
df_house_cdbg.index = range(len(df_house_cdbg))

In [60]:
s_house_id = df_house_cdbg['IDIS Activity ID']

In [61]:
cdbg_notin_house = df_cdbg[~s_cdbg_id.isin(s_house_id)]

In [62]:
house_notin_cdbg = df_house_cdbg[~s_house_id.isin(s_cdbg_id)]

In [63]:
cdbg_notin_house.shape

(173, 10)

In [65]:
house_notin_cdbg.shape # great! only one record... what is it

(1, 14)

In [66]:
house_notin_cdbg

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
563,20726,NaN,514 PINE AVE,2017,22,Commercial Rehabilitation,1547,Open,CDBG,0.0,0.0,0.0,NaN,NaN


#### interesting that the project is open... let's ignore this one for now, and just match the ones that we can

In [72]:
df_cdbg.shape #indexes are same

(686, 10)

In [71]:
s_cdbg_id.shape

(686,)

In [73]:
df_cdbg.head()

,Address,Description,FundedAmount,IDISActivityID,IDISProjectId,InitialFundingDate,PlanYear,Project,Status,StatusDate
0,SANDS DRIVE,,0.00,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
1,421 W SOCIETY AVE,,124559.66,522,11,08/22/2007,2006,DEMOLITION,Completed,4/6/2010
2,525 5TH AVE,,15016.43,523,8,08/22/2007,2006,PUBLIC FACILITIES,Completed,8/16/2011
3,PO BOX 448,DAYCARE SERVICES FOR SENIORS DIAGNOSED WITH AL...,14950.95,539,15,10/04/2007,2007,PUBLIC SERVICE,Completed,8/6/2010
4,400 EBONY LN,Drainage and street improvements made to the 4...,46642.04,546,11,10/04/2007,2007,PUBLIC FACILITIES,Completed,11/29/2010


In [108]:
df_house_cdbg.iloc[0]['IDIS Activity ID']

578

In [116]:
(578 in s_cdbg_id.values)

True

In [160]:
df_cdbg[s_cdbg_id == 578]['Address'].index[0]

473

In [161]:
def match_ids(df_main, df_info, s_info): #potentially reusable but not really. main is full house, info is cdbg
    s_house_addr = []
    s_cdbg_addr = []
    s_initial_date = []
    s_status_date = []
    s_cdbg_ind = []
    for i in range(len(df_main)):
        row = df_main.iloc[i]
        proj_id = row['IDIS Activity ID']
        if proj_id in s_info.values:
            record = df_info[s_info == proj_id] # brings the specific record where there is a match (should be unique)
            ind = record.index[0]
            s_cdbg_ind.append(ind)
            record = record.iloc[0]
            s_house_addr.append(row['Address'])
            s_cdbg_addr.append(record['Address'])
            s_initial_date.append(record['InitialFundingDate'])
            s_status_date.append(record['StatusDate'])
    return pd.DataFrame({'HOUSE_ADDR' : s_house_addr, 'CDBG_ADDR' : s_cdbg_addr, 
                        'INIT_DATE' : s_initial_date, 'STAT_DATE' : s_status_date,
                        'CDBG_INDEX' : s_cdbg_ind})

In [162]:
df_toadd = match_ids(df_house_cdbg, df_cdbg, s_cdbg_id)

In [163]:
df_toadd.shape

(563, 5)

In [164]:
df_toadd.head(5)

,HOUSE_ADDR,CDBG_ADDR,INIT_DATE,STAT_DATE,CDBG_INDEX
0,1008 E RESIDENCE AVE,1008 E RESIDENCE AVE,03/08/2008,4/17/2008,473
1,1100 SWIFT ST,1100 SWIFT ST,06/03/2008,6/30/2008,504
2,1117 W WADDELL AVE,1117 W WADDELL AVE,06/02/2008,6/30/2008,497
3,1223 E BROAD AVE,1223 E BROAD AVE,03/08/2008,4/16/2008,470
4,1316 E 4TH AVE,1316 E 4TH AVE,06/03/2008,4/9/2009,160


In [168]:
mismatched = df_toadd[df_toadd['HOUSE_ADDR'] != df_toadd['CDBG_ADDR']]

In [199]:
mismatched.head()

,HOUSE_ADDR,CDBG_ADDR,INIT_DATE,STAT_DATE,CDBG_INDEX
5,1419 W HIGHLAND AVE,1419 HIGHLAND AVE,05/30/2008,6/30/2008,493
39,705 16TH AVE,706 16TH AVE,06/05/2008,6/30/2008,164
41,UNKNOWN ADDRESS,SCATTERED SITES,10/09/2007,9/29/2009,10
43,230 S JACKSON ST,ALBANY COMMUNITY TOGETHER! (ACT!),06/10/2008,9/29/2009,19
44,314 N JEFFERSON ST,PO BOX 448,10/04/2007,8/6/2010,3


In [172]:
df_house_cdbg.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
0,26627,NaN,1008 E RESIDENCE AVE,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.0,NaN,NaN
1,25739,NaN,1100 SWIFT ST,2007,5,Energy Efficiency,619,Completed,CDBG,4580.04,4580.04,0.0,NaN,NaN
2,21679,NaN,1117 W WADDELL AVE,2007,5,Energy Efficiency,607,Completed,CDBG,3977.52,3977.52,0.0,NaN,NaN
3,5461,NaN,1223 E BROAD AVE,2007,7,Emergency Repairs,575,Completed,CDBG,4776.14,4776.14,0.0,NaN,NaN
4,25773,NaN,1316 E 4TH AVE,2007,7,Emergency Repairs,613,Completed,CDBG,6934.40,6934.40,0.0,NaN,NaN


In [173]:
df_cdbg.head()

,Address,Description,FundedAmount,IDISActivityID,IDISProjectId,InitialFundingDate,PlanYear,Project,Status,StatusDate
0,SANDS DRIVE,,0.00,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
1,421 W SOCIETY AVE,,124559.66,522,11,08/22/2007,2006,DEMOLITION,Completed,4/6/2010
2,525 5TH AVE,,15016.43,523,8,08/22/2007,2006,PUBLIC FACILITIES,Completed,8/16/2011
3,PO BOX 448,DAYCARE SERVICES FOR SENIORS DIAGNOSED WITH AL...,14950.95,539,15,10/04/2007,2007,PUBLIC SERVICE,Completed,8/6/2010
4,400 EBONY LN,Drainage and street improvements made to the 4...,46642.04,546,11,10/04/2007,2007,PUBLIC FACILITIES,Completed,11/29/2010


### Things to investigate

In [190]:
col1 = list(df_house_cdbg)

In [191]:
col2 = list(df_cdbg)

In [209]:
row1 = df_house_cdbg.iloc[5]

In [210]:
row2 = df_cdbg.loc[493]

In [217]:
pd.concat([row1, row2], axis = 0, ignore_index=False)

PrimaryID                                                         18199
Parcel ID                                                           NaN
Address                                             1419 W HIGHLAND AVE
Plan Year                                                          2007
IDIS Project                                                          5
Project                                               Energy Efficiency
IDIS Activity ID                                                    603
Activity Status                                               Completed
Program                                                            CDBG
Funded Amount                                                   3985.52
Draw Amount                                                     3985.52
Balance                                                               0
Multihome_Type                                                      NaN
Multihome                                                       

In [232]:
conflict_funding = pd.DataFrame(columns=col1+col2)
for house_ind, cdbg_ind in zip(mismatched.index, mismatched['CDBG_INDEX']):
    house_row = df_house_cdbg.iloc[house_ind]
    cdbg_row = df_cdbg.loc[cdbg_ind]
    if house_row['Funded Amount'] != cdbg_row['FundedAmount']:
        full = pd.concat([house_row, cdbg_row], axis=0)
        conflict_funding = conflict_funding.append(full, ignore_index=True)
conflict_funding.drop(['Parcel ID', 'IDIS Project', 'Program', 'PrimaryID'], axis=1, inplace=True)

In [233]:
conflict_funding

,Address,Plan Year,Project,IDIS Activity ID,Activity Status,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,Address,Description,FundedAmount,IDISActivityID,IDISProjectId,InitialFundingDate,PlanYear,Project,Status,StatusDate
0,1300 NEWTON RD,2014,Disposition,1228,Completed,12509.81,12509.81,0.0,NaN,NaN,230 S JACKSON ST STE 315 SUITE 315,,37529.44,1228,4,09/23/2014,2014,DISPOSITION,Completed,6/30/2015
1,1306 NEWTON RD,2014,Disposition,1228,Completed,12509.81,12509.81,0.0,NaN,NaN,230 S JACKSON ST STE 315 SUITE 315,,37529.44,1228,4,09/23/2014,2014,DISPOSITION,Completed,6/30/2015
2,2423 HARVEY RD,2014,Disposition,1228,Completed,12509.82,12509.82,0.0,NaN,NaN,230 S JACKSON ST STE 315 SUITE 315,,37529.44,1228,4,09/23/2014,2014,DISPOSITION,Completed,6/30/2015
3,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,42,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
4,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,43,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
5,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,44,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
6,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,45,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
7,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,46,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
8,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,47,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
9,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,48,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
